In [7]:
import pandas as pd
import requests
from datetime import datetime, timedelta

tcbs_headers = {
    'sec-ch-ua':
    '"Google Chrome";v="119", "Chromium";v="119", "Not?A_Brand";v="24"',
    'DNT': '1',
    'Accept-language': 'vi',
    'sec-ch-ua-mobile': '?0',
    'User-Agent':
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36',
    'Content-Type': 'application/json',
    'Accept': 'application/json',
    'Referer': 'https://tcinvest.tcbs.com.vn/',
    'sec-ch-ua-platform': '"Windows"'
}

class TCBS():
    def __init__(self, symbol, asset_type, headers=tcbs_headers):
        self.symbol = symbol
        self.asset_type = asset_type
        self.headers = headers
        self.Candles = self.Candles(self)

    class Candles():
        def __init__(self, tcbs_object):
            self.tcbs_object = tcbs_object 

        def download(self, start_time, end_time, resolution, decor=True, format='df'): 
            start_date = datetime.strptime(start_time, '%Y-%m-%d')
            end_date = datetime.strptime(end_time, '%Y-%m-%d')
            delta = (end_date - start_date).days

            df = pd.DataFrame()

            if resolution == '1D':
                self.resolution = 'D'
            else:
                self.resolution = resolution

            # Batching (Illustrative - adapt based on TCBS API limits)
            max_days_per_request = 365  

            while delta > 0:
                days_to_fetch = min(delta, max_days_per_request)
                end_date_stp = int(end_date.timestamp())
                url = self._construct_url(end_date_stp, days_to_fetch)

                response = requests.request("GET", url, headers=self.tcbs_object.headers)
                status_code = response.status_code

                if status_code == 200:
                    data = response.json()
                    df_temp = pd.DataFrame(data['data'])
                    df_temp['time'] = pd.to_datetime(df_temp['tradingDate']).dt.strftime('%Y-%m-%d')
                    df_temp.drop('tradingDate', axis=1, inplace=True)
                    df = pd.concat([df_temp, df], ignore_index=True)
                else:
                    print(f'Error {status_code}. {response.text}')

                delta -= days_to_fetch

            # Filter data for the specific date range
            df = df[(df['time'] >= start_date.strftime('%Y-%m-%d')) & (df['time'] <= end_date.strftime('%Y-%m-%d'))]

            # Data formatting
            if self.tcbs_object.asset_type == 'stock':
                df[['open', 'high', 'low', 'close']] = round(df[['open', 'high', 'low', 'close']] / 1000, 2)

            df[['open', 'high', 'low', 'close']] = df[['open', 'high', 'low', 'close']].astype(float)
            df['volume'] = df['volume'].astype(int)
            df = df[['time', 'open', 'high', 'low', 'close', 'volume']] 

            return df

        def _construct_url(self, end_date_stp, delta=None):
            if delta is None:
                delta = 365

            if self.tcbs_object.asset_type in ['stock', 'index']:
                return f"https://apipubaws.tcbs.com.vn/stock-insight/v2/stock/bars-long-term?ticker={self.tcbs_object.symbol}&type={self.tcbs_object.asset_type}&resolution={self.resolution}&to={end_date_stp}&countBack={delta}"
            elif self.tcbs_object.asset_type == 'derivative':
                return f'https://apipubaws.tcbs.com.vn/futures-insight/v2/stock/bars-long-term?ticker={self.tcbs_object.symbol}&type=derivative&resolution={self.tcbs_object.resolution}&to={end_date_stp}&countBack={delta}'

In [8]:
# Doesn't work for now
TCB = TCBS(symbol='TCB', asset_type='stock')
df = TCB.Candles.download(start_time='2023-01-02', end_time='2024-01-10', resolution='1D')
df

,time,open,high,low,close,volume
0,2023-12-29,31.70,31.95,31.50,31.8,4233088
1,2024-01-02,31.90,32.25,31.80,32.1,3711936
2,2024-01-03,31.90,32.60,31.85,32.6,5612090
3,2024-01-04,32.60,33.65,32.55,33.1,10076237
4,2024-01-05,33.10,33.40,33.00,33.2,3571390
...,...,...,...,...,...,...
253,2024-01-04,32.60,33.65,32.55,33.1,10076237
254,2024-01-05,33.10,33.40,33.00,33.2,3571390
255,2024-01-08,33.50,34.15,33.35,34.1,6326633
256,2024-01-09,34.00,34.50,33.80,34.5,6902873


In [9]:
VNINDEX = TCBS(symbol='VNINDEX', asset_type='index') # SAI
df = TCB.Candles.download(start_time='2023-01-02', end_time='2024-01-10', resolution='1D', format='df')
df

,time,open,high,low,close,volume
0,2023-12-29,31.70,31.95,31.50,31.8,4233088
1,2024-01-02,31.90,32.25,31.80,32.1,3711936
2,2024-01-03,31.90,32.60,31.85,32.6,5612090
3,2024-01-04,32.60,33.65,32.55,33.1,10076237
4,2024-01-05,33.10,33.40,33.00,33.2,3571390
...,...,...,...,...,...,...
253,2024-01-04,32.60,33.65,32.55,33.1,10076237
254,2024-01-05,33.10,33.40,33.00,33.2,3571390
255,2024-01-08,33.50,34.15,33.35,34.1,6326633
256,2024-01-09,34.00,34.50,33.80,34.5,6902873
